In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

# indlæs data: husk at ændre så det passer din filplacering
mappe_sti = r"C:\Users\kftro\OneDrive - Aarhus universitet\Dataprojekt\Analyse"

# Liste over månederne (skal matche filnavnene)
maaneder = ['Jan', 'Feb', 'Mar', 'Apr', 'Maj', 'Jun', 
            'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dec']



# INDLÆS ALLE 12 MÅNEDER
alle_dataframes = []

print("Starter indlæsning af hele året...")

for m in maaneder:
    # Konstruer filnavnet: "PLC, Product detaljeret, Aarhus, Jan. 2023.xlsx"
    filnavn = f"PLC, Product detaljeret, Aarhus, {m}. 2023.xlsx"
    fuld_sti = os.path.join(mappe_sti, filnavn)
    
    if os.path.exists(fuld_sti):
        try:
            # Prøv som Excel
            temp_df = pd.read_excel(fuld_sti, header=2)
        except:
            # Fallback til CSV
            temp_df = pd.read_csv(fuld_sti, header=2, sep=';', encoding='latin1')
            
        # Gem hvilken måned data kommer fra (kan være brugbart senere)
        temp_df['Måned'] = m
        alle_dataframes.append(temp_df)
        print(f"  -> {m} indlæst.")
    else:
        print(f"  -> {m} fil mangler/kunne ikke findes.")

# Saml alt i én stor tabel
if alle_dataframes:
    df_total = pd.concat(alle_dataframes, ignore_index=True)
    print(f"\nSucces! Har samlet data for {len(df_total)} kasserede artikler.")
else:
    print("Fejl: Ingen data fundet.")
    exit()



Starter indlæsning af hele året...
  -> Jan indlæst.
  -> Feb indlæst.
  -> Mar indlæst.
  -> Apr indlæst.
  -> Maj indlæst.
  -> Jun indlæst.
  -> Jul indlæst.
  -> Aug indlæst.
  -> Sep indlæst.
  -> Okt indlæst.
  -> Nov indlæst.
  -> Dec indlæst.

Succes! Har samlet data for 248711 kasserede artikler.


In [4]:
def find_kategori(navn):
    if not isinstance(navn, str): return "Ukendt"
    navn_ren = navn.lower().replace(',', '').replace('.', '')
    
    # 1. SPECIALBEKLÆDNING (Skal fanges først!) 
    if 'term' in navn_ren: return 'Termotøj'
    if 'kedeldr' in navn_ren: return 'Kedeldragt'
    if 'overall' in navn_ren: return 'Overalls'
    
    # Flyttet op: Hvis det er kokketøj (kokkejakke/kokkebuks, osv.) fanges det her
    if 'kok' in navn_ren: return 'Kokketøj' 
    
    # 2. UNDERDELE 
    if 'underbuk' in navn_ren: return 'Underbukser'
    if any(x in navn_ren for x in ['ben', 'buks', 'jeans', 'unisexben']): return 'Bukser'
    if 'shorts' in navn_ren: return 'Shorts'
    
    # 3. OVERDELE
    if 'cardigan' in navn_ren: return 'Cardigan'
    if 'sweat' in navn_ren: return 'Sweatshirt'

    if any(x in navn_ren for x in ['t-shirt', 'tshirt']): return 'T-shirt' 

    if 'polo' in navn_ren: return 'Polo'
    if 'hætte' in navn_ren: return 'Hættetrøje'
    if 'skj' in navn_ren: return 'Skjorte'
    if 'bussero' in navn_ren: return 'Busseronne'
    if 'tunika' in navn_ren: return 'Tunika'
    if 'kit' in navn_ren: return 'Kittel'
    if 'vest' in navn_ren: return 'Vest'
    
   
    # Denne linje fanger kun jakker, der IKKE eksempelvis er kokkejakker eller termojakker. 
    if any(x in navn_ren for x in ['jak', 'jk']): return 'Jakke'
    
    if 'fleece' in navn_ren: return 'Fleece'
    if 'kasket' in navn_ren: return 'Kasket'
    
    # 4. KØKKEN & TILBEHØR
    if 'forklæde' in navn_ren or 'smæk' in navn_ren: return 'Forklæde'

    
    # Opsamling
    parts = navn_ren.split()
    if len(parts) > 1:
        return "Andet (" + parts[1].title() + ")"
    return "Andet"

In [ ]:
# Tjek de N mest populære kategorier

N_kategori = 50

df_total['Kategori'] = df_total['Produkt - Produkt'].apply(find_kategori)

# Tjek resultatet - nu burde listen være meget renere
print(df_total['Kategori'].value_counts().head(N_kategori))

Kategori
Bukser                   76946
T-shirt                  73433
Skjorte                  17690
Jakke                    13473
Kokketøj                 10671
Overalls                 10669
Sweatshirt                6631
Busseronne                5600
Fleece                    4670
Vest                      4186
Termotøj                  3531
Forklæde                  3167
Kittel                    2868
Shorts                    2023
Tunika                    1794
Polo                      1554
Cardigan                  1186
Kasket                     951
Andet (Pull)               902
Kedeldragt                 563
Andet (Softshell)          557
Andet (Mulepose)           353
Andet (Nederdel)           341
Andet (Rullekrave)         277
Andet (Hat)                277
Andet (Heldragt)           265
Andet (Sjælevarmer)        253
Andet (3F)                 249
Andet (Arbsko)             230
Andet (Trekking)           191
Andet (Dame)               188
Andet (Undertrøje)         179

In [9]:
# Tjek hvordan hvert unikke produkt kategoriseres

# 1. Gruppér efter Produktnavn og Kategori, og tæl antallet
kategori_tjek = df_total.groupby(['Produkt - Produkt', 'Kategori']).size().reset_index(name='Antal')

# 2. Sorter så de mest populære produkter står øverst
kategori_tjek = kategori_tjek.sort_values(by='Antal', ascending=False)

# 3. Vis resultatet
# Vi bruger pd.set_option for at sikre, at vi kan se hele listen (eller i hvert fald mere af den)
pd.set_option('display.max_rows', 100) 
print(kategori_tjek.head(50)) # Vis de top 50 produkter

                       Produkt - Produkt    Kategori  Antal
496   09385 T-Shirt Hvid B/P M Arla Tryk     T-shirt   7196
1367    23191 Benkl Mind pullon m opsmøg      Bukser   7063
27     01083 T-Shirt unisex hvid k/æ P/B     T-shirt   6776
47         01589 T-shirt unisex mblå k/æ     T-shirt   6694
891   13084 T-shirt prowear dame navy pb     T-shirt   6152
1121     14152 Benkl m/lom hvid low risk      Bukser   5062
1855      28793 Benkl M/Lom Grå Low Risk      Bukser   4434
765   12143 Pull on let buks opsmøg mblå      Bukser   4256
1540    24781 Flex buks da CSR dark rock      Bukser   4164
532                   09813 T-Shirt Vask     T-shirt   3781
254           05257 T-Shirt Pro Sort K/Æ     T-shirt   3099
1297  22739 Funktionsbuks m ela/liv sort      Bukser   3026
1076    13907 Unisexben Lårl & Toml sort      Bukser   3020
1054     13819 Skjorte funktion dame Sky     Skjorte   2995
1095    13976 T-shirt l.ærme m Arla logo     T-shirt   2714
1331  23016 Skjorte funkt da Sky mørk kr

In [11]:
# Opret en excelfil med alle unikke produkter

# nyttigt for at tjekke igennem for fejl i kategoriseringen
kategori_tjek.to_excel("Kategori_Overblik.xlsx", index=False)

In [12]:
# Vis kun produkterne der er kategoriseret som noget med "Andet"
andet_fejl = kategori_tjek[kategori_tjek['Kategori'].str.contains("Andet")]
print(andet_fejl.head(20))

                       Produkt - Produkt             Kategori  Antal
1580     25179 Pull on uni CSR dark rock         Andet (Pull)    900
57    01754 Rullekrave Bluse Hvid Bomuld   Andet (Rullekrave)    277
1299      22775 Hat med net fl.gul AMCOR          Andet (Hat)    277
655             11343 Sjælevarmer marine  Andet (Sjælevarmer)    251
1569        25116 Mulepose HKScan marine     Andet (Mulepose)    249
398           08099 3F Funktionssok Sort           Andet (3F)    246
1364               23188 Nederdel Cheans     Andet (Nederdel)    192
391              07990 Trekking Sok Mblå     Andet (Trekking)    191
1769   27362 Dame Fusion Shirt blue dark         Andet (Dame)    172
386             07961 Strømper Tynd sort     Andet (Strømper)    164
1369                 23193 Nederdel Mind     Andet (Nederdel)    148
1373         23197 Rygsæk 2 rum 40L sort       Andet (Rygsæk)    134
403        08190 Heldragt Blå Alsistrong     Andet (Heldragt)    128
933         13345 Halsedisse Kalma